<a href="https://colab.research.google.com/github/wittysean/COMPUTER-PROGRAMMING-AND-APPLICATION-113-2/blob/main/MJ_LINEBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install line-bot-sdk ngrok pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 10.8 MB/s eta 0:00:00


In [4]:

from google.colab import userdata
from pyngrok import ngrok
from linebot.v3.webhook import WebhookHandler
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import MessagingApi, Configuration, TextMessage, ReplyMessageRequest
from linebot.v3.messaging.models import TemplateMessage, CarouselTemplate, CarouselColumn, MessageAction
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import ApiClient

# Colab secrets for credentials
CHANNEL_ACCESS_TOKEN = userdata.get("LINE_CHANNEL_ACCESS_TOKEN")
CHANNEL_SECRET = userdata.get("LINE_CHANNEL_SECRET")
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")


In [5]:

# Apply ngrok authentication
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


In [12]:

album_data = {
    "Got to Be There": {
        "year": 1972, "sales": 2000000,
        "songs": [
            "Ain't No Sunshine", "I Wanna Be Where You Are", "Girl Don't Take Your Love From Me",
            "In Our Small Way", "Got to Be There", "Rockin' Robin", "Wings of My Love",
            "Maria (You Were the Only One)", "Love Is Here and Now You're Gone", "You've Got a Friend"
        ],
        "cover": "https://upload.wikimedia.org/wikipedia/en/thumb/b/b2/Mj1971-got-to-be-there.jpg/250px-Mj1971-got-to-be-there.jpg"
    },
    "Ben": {
        "year": 1972, "sales": 2000000,
        "songs": [
            "Ben", "Greatest Show on Earth", "People Make the World Go 'Round", "We've Got a Good Thing Going",
            "Everybody's Somebody's Fool", "My Girl", "What Goes Around Comes Around", "In Our Small Way",
            "Shoo-Be-Doo-Be-Doo-Da-Day", "You Can Cry on My Shoulder"
        ],
        "cover": "https://lh3.googleusercontent.com/qx2XfantCerpCCvMMJRHWulhaj73WDavPph7aB19EG-mDsbIpRk6prjQxVhl6BOmX1RVFP_l-uoNX_Akyg=w544-h544-s-l90-rj"
    },
    "Music & Me": {
        "year": 1973, "sales": 1000000,
        "songs": [
            "With a Child's Heart", "Up Again", "All the Things You Are", "Happy", "Too Young",
            "Doggin' Around", "Johnny Raven", "Euphoria", "Morning Glow", "Music and Me"
        ],
        "cover": "https://lh3.googleusercontent.com/33_r_7savr68acaP1Df50BpQyv_ZO3uZIMT-BAASqmswcXF_9xXupUifxNzxY6Ia1nXyL5VmOIfvxgAhDA=w544-h544-l90-rj"
    },
    "Forever, Michael": {
        "year": 1975, "sales": 1000000,
        "songs": [
            "We're Almost There", "Take Me Back", "One Day in Your Life", "Cinderella Stay Awhile",
            "We've Got Forever", "Just a Little Bit of You", "You Are There", "Dapper-Dan",
            "Dear Michael", "I'll Come Home to You"
        ],
        "cover": "https://lh3.googleusercontent.com/CCP03PdfzD_z30UlbwV6fqAoPivZZNH445W-avgxcx63hrCHZlXNDul3Ze9Sd0OVVbn0Ic1dH0FSMYQjsQ=w544-h544-s-l90-rj"
    },
    "Thriller": {
        "year": 1982, "sales": 70000000,
        "songs": [
            "Wanna Be Startin' Somethin'", "Baby Be Mine", "The Girl Is Mine", "Thriller",
            "Beat It", "Billie Jean", "Human Nature", "P.Y.T. (Pretty Young Thing)", "The Lady in My Life"
        ],
        "cover": "https://upload.wikimedia.org/wikipedia/en/5/55/Michael_Jackson_-_Thriller.png"
    },
    "Bad": {
        "year": 1987, "sales": 35000000,
        "songs": [
            "Bad", "The Way You Make Me Feel", "Speed Demon", "Liberian Girl", "Just Good Friends",
            "Another Part of Me", "Man in the Mirror", "I Just Can't Stop Loving You", "Dirty Diana",
            "Smooth Criminal", "Leave Me Alone"
        ],
        "cover": "https://upload.wikimedia.org/wikipedia/en/5/51/Michael_Jackson_-_Bad.png"
    },
    "Dangerous": {
        "year": 1991, "sales": 32000000,
        "songs": [
            "Jam", "Why You Wanna Trip on Me", "In the Closet", "She Drives Me Wild", "Remember the Time",
            "Can't Let Her Get Away", "Heal the World", "Black or White", "Who Is It", "Give In to Me",
            "Will You Be There", "Keep the Faith", "Gone Too Soon", "Dangerous"
        ],
        "cover": "https://lh3.googleusercontent.com/acFvHA1OEoI0HBPPG33zidd9n9aG1OTvo7XQQeFjEeQObGv6R3464BvFijHerp3Sit5UeHvQnx6LMoE=w544-h544-l90-rj"
    },
    "HIStory": {
        "year": 1995, "sales": 20000000,
        "songs": [
            "Scream", "They Don't Care About Us", "Stranger in Moscow", "This Time Around", "Earth Song",
            "D.S.", "Money", "Come Together", "You Are Not Alone", "Childhood", "Tabloid Junkie",
            "2 Bad", "HIStory", "Little Susie", "Smile"
        ],
        "cover": "https://lh3.googleusercontent.com/R7pwf7-lcPvK3dxv8jMkUd4SlbVmrM-nZOFEJqnHGLQFBl4lqj1gyeWSFO5X9HZxgUhTh4KM8n0l7j_k=w544-h544-l90-rj"
    },
    "Invincible": {
        "year": 2001, "sales": 6000000,
        "songs": [
            "Unbreakable", "Heartbreaker", "Invincible", "Break of Dawn", "Heaven Can Wait", "You Rock My World",
            "Butterflies", "Speechless", "2000 Watts", "You Are My Life", "Privacy", "Don't Walk Away",
            "Cry", "The Lost Children", "Whatever Happens", "Threatened"
        ],
        "cover": "https://lh3.googleusercontent.com/L0nwBz3JF8kFa1-1PAOiLC6pB49lsRc3QRCbj1gvU3uCNf4SXnLUGtAFBha1CZ02mhQzMJrcl-rJ0pSG=w544-h544-l90-rj"
    },
    "Michael": {
        "year": 2010, "sales": 1000000,
        "songs": [
            "Hold My Hand", "Hollywood Tonight", "Keep Your Head Up", "I Like the Way You Love Me",
            "Monster", "Best of Joy", "Breaking News", "(I Can't Make It) Another Day", "Behind the Mask", "Much Too Soon"
        ],
        "cover": "https://lh3.googleusercontent.com/-DE1fXN4FdvCXTfQz9N6TMX1qhjyIOYO5rRlFpWpZxBPWLH2un4Ia_yAuuZwXgtNEJPdeWvW5cBfL_ct=w544-h544-l90-rj"
    },
    "Xscape": {
        "year": 2014, "sales": 1000000,
        "songs": [
            "Love Never Felt So Good", "Chicago", "Loving You", "A Place with No Name", "Slave to the Rhythm",
            "Do You Know Where Your Children Are", "Blue Gangsta", "Xscape"
        ],
        "cover": "https://lh3.googleusercontent.com/qk8rEQ5zgdeXxszR37SjJHOPOlPG6NhD5F4r-rxdxdrdmf7rF6xv_OdrCBuZBxaTFc4fVK2ImJLW5FKt=w544-h544-l90-rj"
    }
}
vote_counter = {album: 0 for album in album_data}
song_lookup = {song.lower(): album for album, data in album_data.items() for song in data["songs"]}


In [7]:

from flask import Flask, request, abort

app = Flask(__name__)
configuration = Configuration(access_token=CHANNEL_ACCESS_TOKEN)
api_client = ApiClient(configuration)
line_bot_api = MessagingApi(api_client)
handler = WebhookHandler(CHANNEL_SECRET)

def format_sales(sales):
    return f"{sales:,} ({sales//1_000_000}M)"

def build_album_carousel():
    columns = []
    for album_name, data in album_data.items():
        column = CarouselColumn(
            thumbnail_image_url=data['cover'],
            title=album_name,
            text=f"{data['year']} - {format_sales(data['sales'])}",
            actions=[
                MessageAction(label="View Info", text=f"info {album_name}"),
                MessageAction(label="Vote", text=f"vote {album_name}")
            ]
        )
        columns.append(column)
    carousel_template = CarouselTemplate(columns=columns[:10])
    return TemplateMessage(alt_text="Album Selection", template=carousel_template)

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=False).decode('utf-8')
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent)
def handle_message(event):
    if isinstance(event.message, TextMessageContent):
        user_text = event.message.text.strip()

        if user_text.lower() == 'menu':
            carousel_msg = build_album_carousel()
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[carousel_msg])
            )
            return

        if user_text.lower().startswith("info "):
            album_name = user_text[5:].strip()
            if album_name in album_data:
                data = album_data[album_name]
                songs = "\n".join(f"- {song}" for song in data["songs"])
                reply = f"🎶 Album: {album_name}\n📅 Year: {data['year']}\n💿 Sales: {format_sales(data['sales'])}\n🎵 Songs:\n{songs}"
            else:
                reply = "❌ Album not found."
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[TextMessage(text=reply)])
            )
            return

        if user_text.lower().startswith("vote "):
            album_name = user_text[5:].strip()
            if album_name in vote_counter:
                vote_counter[album_name] += 1
                reply = f"✅ Voted for {album_name}! Total votes: {vote_counter[album_name]}"
            else:
                reply = "❌ Album not found for voting."
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[TextMessage(text=reply)])
            )
            return

        if user_text.lower() == "/top":
            sorted_votes = sorted(vote_counter.items(), key=lambda x: x[1], reverse=True)
            leaderboard = "\n".join(f"{i+1}. {name} - {votes} votes" for i, (name, votes) in enumerate(sorted_votes))
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[TextMessage(text=leaderboard)])
            )
            return

        song_key = user_text.lower()
        if song_key in song_lookup:
            album_name = song_lookup[song_key]
            data = album_data[album_name]
            songs = "\n".join(f"- {song}" for song in data["songs"])
            reply = f"🎶 Album (from song '{user_text}'): {album_name}\n📅 Year: {data['year']}\n💿 Sales: {format_sales(data['sales'])}\n🎵 Songs:\n{songs}"
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[TextMessage(text=reply)])
            )
            return

        reply = "🤖 Type 'menu' to browse albums, or enter song name."
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(reply_token=event.reply_token, messages=[TextMessage(text=reply)])
        )


In [ ]:

public_url = ngrok.connect(5000).public_url
print("Your public URL to paste into LINE webhook:", public_url + "/callback")
app.run(port=5000)


Your public URL to paste into LINE webhook: https://b047-34-48-183-192.ngrok-free.app/callback
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 19:19:24] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 19:19:38] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 19:19:42] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 19:19:49] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 19:19:55] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 19:22:29] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 19:22:33] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 19:22:39] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 19:22:52] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 19:22:58